### TMA4320 - Project 2: Physics


# 0.0 Motivation

In [ ]:

# Importing libraries
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from matplotlib import animation
from tempfile import TemporaryFile


# To show figures in the notebook
%matplotlib inline

# Make the figures more readable
newparams = {'figure.figsize': (18, 9), 'axes.grid': False,
             'lines.linewidth': 2,
             'font.size': 25, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
# figure.figsize sets the size of the figure (values are in inches)
# axes.grid, set to False means that there is no grid
# lines.linewidth sets the thickness of the lines in the plot
# font.size sets the size of the text (ticks, title, xlabel, etc.)
# mathtext.fontset sets the font of the text of the type r'$tekst$' (math text)
# font.family sets the font of ordinary text (ticks, title, xlabel, etc.)

# There are several other parameters you can change, and you can look them up in the documentation 
# for matplotlib if you need to change any. 

plt.rcParams.update(newparams) # Updates the parameter set above. 


In [ ]:
# Parameters
sigma_0 = 1000 # kg/m^2
sigma = 500 # kg/m^2
R = 10 # m, Radius
A_s = 0.5*np.pi*(R**2) # Cross-section of boat
g = 9.81 # m/s^2
h = (4*R)/(3*np.pi)
m_boat = A_s*sigma

A_0 = (sigma*np.pi*(R**2))/(2*sigma_0)


For å undersøke båtens endring i hellingsvinkel

In [ ]:
def calculate_I_c(m_load, s_L):
    '''
    This function calculated the ships total rotational inertia of the ship and its load, at the
    relative position of the load affects the total rotational inertia.
    Input:
        m_load: float, the mass of the load [kg]
        s_L:    float, the loads distance from the metacenter [m]
    Output:
        I_c:    float, the ships total rotational inertia [kg/m²]
    '''
    I_boat = 0.5*m_boat*(R**2)*(1-(32/(9*(np.pi**2)))) # Rotational inertia from the ship about C

    r_load = np.sqrt( h**2 + s_L**2 ) # Distance from ship's CM to load's CM
    I_load = m_load*r_load**2   # Rotational inertia from the load about C 
    
    I_c = I_load + I_boat # Use the superposition property of rotational inertias (Steiner's Thm).
    return I_c


# print(calculate_I_c(0.001*m_boat, 0))
# print(calculate_I_c(0.001*m_boat, R))

In [ ]:
def calculate_beta(rhs_constant, x0, tol=1.e-8, max_iter=50):
    ''' Given a right hand side and left hand side of a function and its derivatives, calculates the beta value of the function.
    
    Solve f(x)=0 by Newtons method, 
    
        The output of each iteration is printed
        Input:
        f_lhs: left hand side of function
        df_lhs: left hand side of derivative of function
        f_rhs: right hand side of function
        df_rhs: right hand side of derivative of function
        x0:  Initial values
        tol: The tolerance
      Output:
        The root and the number of iterations
    '''

    f = lambda beta: beta - np.sin(beta) - rhs_constant
    df = lambda beta: 1 - np.cos(beta)


    x = x0
    for k in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:           # Accept the solution 
            break 
        x = x - fx/df(x)            # Newton-iteration
    return x

In [ ]:
def print_beta():
    f_lhs = lambda beta: beta-np.sin(beta)
    df_lhs = lambda beta: 1 - np.cos(beta)

    f_rhs = lambda beta : np.pi * sigma/sigma_0
    df_rhs = 0

    beta = calculate_beta(np.pi*sigma/sigma_0,5)
    print("Beta = ",round(np.degrees(beta), 2),"degrees")
print_beta()

In [ ]:
#Define the global variables for the equilibrium, as requested in 1b)
y_M_0 = lambda beta: R*np.cos(beta/2)
y_C_0 = lambda beta: y_M_0(beta) - 4*R/(3*np.pi)
y_B_0 = lambda beta: y_M_0(beta) - 4*R*(np.sin(beta/2))**3 /(3*(beta-np.sin(beta)))
y_D_0 = lambda beta: y_M_0(beta) - R

#Values as functions of delta y_C and theta. 
y_M = lambda beta, theta, delta_y_C : y_M_0(beta) + ((4*R)/(3*np.pi))*(np.cos(theta)-1) + delta_y_C
y_C = lambda beta,delta_y_c : y_C_0(beta) + delta_y_c
y_D = lambda beta, theta, delta_y_c : y_D_0(beta) + ((4*R)/(3*np.pi))*(np.cos(theta)-1) + delta_y_c
y_B = lambda beta: y_B_0(beta)

#Note: beta is a global variable
gamma = lambda beta, theta, delta_y_c : 2*np.arccos(np.cos(beta/2)-(4/(3*np.pi))*(1-np.cos(theta))+delta_y_c/R)

water_area = lambda gamma_number: 0.5*(R**2)*(gamma_number - np.sin(gamma_number))

F_G = lambda m : -m*g
F_B = lambda A : A*sigma_0*g
friction = lambda k_f, omega, gamma_number : -k_f * R * gamma_number*omega


## 0.? Visualisering av data

For å analysere opparbeidete resultater er det hensiktsmessig å visualisere det i plot. Derfor definerer vi en fleksibel funksjon som plotter datasett på en foretrukket måte. Denne vil bli brukt gjennomgående i rapporten.


# 1.3 Euler metoden


In [ ]:
def euler_step(f, t, w, h_step):
    '''
    Tar inn en vektor-funksjon på formen w = [theta(t), omega(t)], med nåværende tid (t) og 
    tilstand (w), og regner ut ett Euler-steg.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t: float, nåvernde tid
        w: s-array, som beskriver systemet
        h_step: steglengde
    Ut:
        t_new: neste tidssteg (t+h)
        w_new: systemet etter ett beregnet tidssteg    
    '''
    
    w_new = w + h_step*f(t, w)
    return t+h, w_new

In [ ]:
def solve_ODE_old(f, t_0, t_end, w_0, h_step, method):
    '''
    Funksjon som løser diff-liking ved å benytte Euler-metoden.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t_0: float, starttid
        t_end: float, slittid
        w_0: s-array, som systemets startposisjon
        h_step: steglengde
    Ut:
        t_num: array, tidsstegene
        w_num: s-array, de beregnede verdiene for theta of omega    
    '''
    w_num = np.array([w_0])  
    t_num = np.arange(t_0, t_end + h_step, h_step)   
    t,w_n = method(f, t_num[0], w_num[0], h_step)
    w_num = np.concatenate((w_num, np.array([w_n])))
    for i in range(1,len(t_num)-1):#ikke inkl første element
        if t_num[-1] > t_end- 1.0e-10: #For å treffe endepunktet. 10e-10 for å unngå "truncation error"
            t_num[-1] = t_end
        
        t,w_n = method(f, t_num[i], w_num[i], h_step)
        
        w_num = np.concatenate((w_num, np.array([w_n])))
    #print(t_num)
    return t_num, w_num

In [ ]:
# New solve_ode
def solve_ODE(f, t_0, t_end, w_0, h_0, method, fixed_step = True, tol = 1e-6, max_iter = 1e5):
    '''
    Funksjon som løser differensial ligninger på en brukerdefinert måte.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t_0: float, starttid
        t_end: float, slittid
        w_0: s-array, som systemets startposisjon
        h_0: Dersom vi ønsker dynamisk steglengde vil dette være startsteglengden
            ellers blir dette den konstante steglengden
        tol: The tollerance if we wish to use adaptive step size
    Ut:
        t_num: array, tidsstegene
        w_num: s-array, de beregnede verdiene for theta of omega    
    '''
    t_num = np.array([t_0])
    w_num = w_0
    # Set the running values
    t_n = t_0
    w_i = w_0
    h_step = h_0
    

    if fixed_step:
        N = int( (t_end-t_0)/h_step + 1)
        t_num, h_step = np.linspace(t_0, t_end, N, retstep=True)  # Use linpace, as numpys documentation suggests this for
                                            # non-integer values, we also get the closed interval [t_0, t_end].
        # print(f't_num:\n {t_num}\n t_num minus one:\n {t_num[0:-1]}')        

        # This loop will iterate over all the t-values exept the last one. The reason for this is because the w corresponding
        # to the last t-value is calculated at the second last t-value, and so, both arrays have an equal lenght.
        for t_n in t_num[0:-1]: 
            w_i = method(f, t_n, w_i, h_step)[1] # Here we are only interrested in w, as the step size is fixed
            #print(f'w_num:\n {w_num}\n\n w_{i+1}:\n {w_n}\n\n\n\n ')
            w_num = np.concatenate(( w_num, w_i), axis=1)

    else: 
        current_iter = 0
        while t_n < t_end-1e-10: # take truncation error into account        
            if t_n + h_step > t_end:
                h_step = t_end-t_n
                
            # One step with the method:
            t_try, w_try, err_est, p = method(f, t_n, w_n, h_step)
            current_f_calls += 1
            
            if err_est <= tol:
                # Use the solution
                t_n = t_try
                w_n = w_try
                
                # Store the solutions
                t_num = np.append(t_num, t_n)           
                w_num = np.concatenate((w_num, np.array([w_n])))
                
            
            # Adjust the step size, start by defining facors
            P = 0.8 # Pessismist facor, normally between 0.5 and 0.95
            
            h_step = P*( (tol/err_est)**(1/(p+1)) )*h_step

            if current_iter >= max_iter:
                print('Maximum number of function calls are reached.')
                return t_num, w_num

    
    
    
    return t_num, w_num

Equating the...

In [ ]:
def test_euler():
    # f = lambda t, w: np.array([
    #     w[1],
    #     -(A_0 * sigma_0 * g * h)/calculate_I_c(0, 0) * np.sin( w[0] )
    #     ])
    
    # This function is equivalent
    def f(t, w):
        dw = np.array([
            w[1],
            -(A_0 * sigma_0 * g * h)/calculate_I_c(0, 0) * np.sin( w[0] )
        ])
        #print("w_0 that works: ", w[0])
        #print(type(w[0]))
        return dw

    # Define the inital condititons, time intervall, and step size
    theta_0 = np.array([np.radians(20)])
    omega_0 = np.array([0])
    w_0 = np.array([theta_0, omega_0]) # Combine all the initial conditions to the array w

    t_0 = 0
    t_end = 20

    h_step = 0.001  #The step size of our simulation


    # Calculate the angles and angle velocities in the given time intervall using the Euler method
    t_num, w_num = solve_ODE(f, t_0, t_end, w_0, h_step, euler_step)

    theta_num, omega_num = w_num[0:2]
    # print(f'theta:\n {theta_num}\n\nomega:\n{omega_num}')
    

    # Plot the angles vs time
    plt.plot(t_num, np.rad2deg(theta_num), label = "dt = 0.01")
    plt.title("Vinkelutslag som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'$\theta$ [$^\circ$]')   
    # plt.ylim(-0.4,0.4)        
    plt.legend(loc=3, shadow = True)
    plt.show()

# test_euler()

## Small angle approximation
...

In [ ]:
def angle_approx_comparison():
    # def f(t, w):
    #     const = (A_s*sigma * g * h)/calculate_I_c(0, 0)
    #     return np.array([w[1], -const*np.sin(w[0]),0,0,0,0,0,0])

    # def f_small_angle(t, w):
    #     const = (A_s*sigma * g * h)/calculate_I_c(0, 0)
    #     return np.array([w[1], -const*w[0],0,0,0,0,0,0])

    #######################################################################################
    #       Define the functions differential equations that we will be solving
    f = lambda t, w: np.array([
        w[1],
        -(A_s*sigma * g * h)/calculate_I_c(0, 0) * np.sin(w[0])
    ])
    f_small_angle = lambda t, w: np.array([
        w[1],
        -(A_s*sigma * g * h)/calculate_I_c(0, 0) * w[0]
    ])

    #######################################################################################
    #       Define the inital condititons, time intervall, and step size
    theta_0 = np.array([np.radians(20)])
    omega_0 = np.array([0])
    w_0 = np.array([theta_0, omega_0]) # Combine all the initial conditions to the array w

    t_0 = 0
    t_end = 20

    h_step = 0.001  #The step size of our simulation
    #######################################################################################
    #       Solve diff. eqn. by the given methods, and 
    t_num, w_num = solve_ODE(f, t_0, t_end, w_0, h_step, euler_step)
    t_num_small_angle_approx, w_num_small_angle_approx = solve_ODE(f_small_angle, t_0, t_end, w_0, h_step,euler_step)


    theta_num, omega_num = w_num[0:2]
    theta_num_small_angle_approx, omega_num_small_angle_approx = w_num_small_angle_approx[0:2]
    
    diff = theta_num_small_angle_approx - theta_num    


    plt.plot(t_num, theta_num, label = "Ikke småvinkeapproksimasjon")
    plt.plot(t_num_small_angle_approx, theta_num_small_angle_approx, label = "Småvinkeapproksimasjon")
    # plt.plot(t_num, diff, label = "Differanse")
    plt.title("Vinkelutslag som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'$\theta$ [rad]')   
    # plt.ylim(-0.2,0.2)        
    plt.legend( 
        bbox_to_anchor=(0,-0.1),
        loc='upper left',
        shadow = True
        )
    plt.show()

# angle_approx_comparison()

## 1.3 Runge Kutta metoden

Implementerer fjerde ordens Runge Kutta.


In [ ]:
def rk4_step(f, t, w, h):
    '''
    Tar inn en vektor-funksjon på formen f(t, _w_) = _y'_ = _y_ + _g(x)_, med nåværende tid (t) og 
    tilstand (w), og regner ut ett Runge Kutta steg.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t: float, nåvernde tid
        w: s-array, som beskriver systemet
    Ut:
        t_new: neste tidssteg (t+h)
        w_new: systemet etter ett beregnet tidssteg    
    '''

    k_1 = f(t, w)
    k_2 = f(t + 0.5*h, w + h*k_1*0.5)
    k_3 = f(t + 0.5*h, w + h*k_2*0.5)
    k_4 = f(t + 1.0*h, w + h*k_3)

    w_new = w + h*(1/6) * (k_1 + 2*k_2 + 2*k_3 + k_4)
    t_new = t + h
    
    return t_new, w_new




In [ ]:
def test_runge_kutta():

    # def f(t, w):
    #     const = (A_s*sigma * g * h)/calculate_I_c(0, 0)
    #     return np.array([w[1], -const*np.sin(w[0]),0,0,0,0,0,0])
    
    #######################################################################################
    #       Define the functions differential equations that we will be solving
    f = lambda t, w: np.array([
        w[1],
        -(A_0 * sigma_0 * g * h)/calculate_I_c(0, 0) * np.sin( w[0] )
        ])


    #######################################################################################
    #       Define the inital condititons, time intervall, and step size
    theta_0 = np.array([np.radians(20)])
    omega_0 = np.array([0])
    w_0 = np.array([theta_0, omega_0]) # Combine all the initial conditions to the array w

    t_0 = 0
    t_end = 20

    h_step = 0.01  #The step size of our simulation
    #######################################################################################

    t_num_rk4, w_num_rk4 = solve_ODE(f, t_0, t_end, w_0, h_step, rk4_step)
    theta_num_rk4, omega_num_rk = w_num_rk4[0:2]


    plt.plot(t_num_rk4, theta_num_rk4, label = "dt = 0.01")
    plt.title("Vinkelutslag som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'$\theta$ [rad]')   
    plt.ylim(-0.4,0.4)        
    plt.legend(loc=3, shadow = True)
    plt.show()
    

test_runge_kutta()

## 1h)
...


In [ ]:
def anal_vs_num():
    #######################################################################################
    #       Define the functions differential equations that we will be solving
    def f_analytic(t, theta_0):
        return theta_0*np.cos( np.sqrt( (m_boat*g*h)/calculate_I_c(0, 0) )*t )
    
    # To be used in numerical solutions
    f_small_angle = lambda t, w: np.array([
        w[1],
        -(A_0 * sigma_0 * g * h)/calculate_I_c(0, 0) * w[0]
        ])
    f_not_small_angle = lambda t, w: np.array([
        w[1],
        -(A_0 * sigma_0 * g * h)/calculate_I_c(0, 0) * np.sin( w[0] )
        ])
    
    # def f_small_angle(t, w):
    #     const = (A_s*sigma * g * h)/calculate_I_c(0, 0)
    #     return np.array([w[1], -const*w[0],0,0,0,0,0,0])
    # def f_not_small_angle(t, w):
    #     const = (A_s*sigma * g * h)/calculate_I_c(0, 0)
    #     return np.array([w[1], -const*  np.sin(w[0]),0,0,0,0,0,0])


    # theta_0 = np.radians(0.01)
    # omega_0 = 0
    # w_0 = np.array([theta_0, omega_0,0,0,0,0,0,0])

    # t_0 = 0
    # t_end = 20


    #######################################################################################
    #       Define the inital condititons, time intervall, the array of step sizes, 
    #       and the analytic value at t=20s
    theta_0 = np.array([0.01])
    omega_0 = np.array([0])
    w_0 = np.array([
        theta_0, 
        omega_0
    ]) # Combine all the initial conditions to the array w

    t_0 = 0
    t_end = 20

    h_steps = 10**np.linspace(-4, -1, 30)


    #######################################################################################
    #       Find the values of different methods (including analytic) at t=20s
    #       using different step sizes
    f_analytic_t_20 = f_analytic(t_end, theta_0) # Analytic value at t=20s

    theta_t20_arr_euler = np.array([])
    theta_t20_arr_rk4 = np.array([])

    # Iterate through all the desired step sizes, and store them
    for h_step in h_steps:
        theta_euler    =   solve_ODE(f_small_angle, t_0, t_end, w_0, h_step, euler_step)[1][0] #[1]: w, [0]: theta
        theta_rk4      =   solve_ODE(f_small_angle, t_0, t_end, w_0, h_step, rk4_step)[1][0]
        
        # t_euler, theta_euler = solve_ODE(f_small_angle, t_0, t_end, w_0, h_step, euler_step) #[1]: w, [0]: theta)
        # t_rk4, theta_rk4 = solve_ODE(f_small_angle, t_0, t_end, w_0, h_step, euler_step) #[1]: w, [0]: theta)
        # print(f'last euler t: {t_euler[-1]}')
        # print(f'last rk4 t: {t_rk4[-1]}')

        # Store the angle at t=20s (the last angle)
        theta_t20_arr_euler =   np.append(theta_t20_arr_euler, theta_euler[-1])
        theta_t20_arr_rk4   =   np.append(theta_t20_arr_rk4 , theta_rk4[-1])


    difference_euler    = theta_t20_arr_euler   -   f_analytic_t_20
    difference_rk4      = theta_t20_arr_rk4     -   f_analytic_t_20
    # difference_rk4 = np.abs(difference_rk4)
    
    #######################################################################################
    #       Save the data to a file
    np.savez(
        'simulation_data/difference_at_t20',
        h_steps = h_steps,
        difference_euler = difference_euler,
        difference_rk4 = difference_rk4
        
        )
# anal_vs_num()

In [ ]:
#######################################################################################
#       Plot the differences
def plot_difference():
    
    diff_data = np.load('simulation_data/difference_at_t20.npz')
    h_steps, difference_euler, difference_rk4 = diff_data['h_steps'], diff_data['difference_euler'], diff_data['difference_rk4']

    plt.yscale("log")
    plt.xscale("log")
    plt.plot(h_steps, np.zeros(len(h_steps)),linestyle='--') # Stiplet linje i 0
    plt.plot(h_steps, np.abs(difference_euler), label = "Euler")
    plt.plot(h_steps, np.abs(difference_rk4), label = "RK4")
    plt.plot(h_steps, np.abs(difference_rk4),'o', label = "Logaritmisk linspace")

    plt.title("Differansen mellom den analytiske og RK4 ved t = 20 s")
    plt.xlabel('$h$')           
    plt.ylabel(r'$\Delta\theta$ [rad]')   
    #plt.ylim(-0.2,0.2)        
    plt.legend(loc=1, shadow = True)
    plt.show()


    # plt.yscale("log")
    # plt.plot(h_steps, np.zeros(len(h_steps)),linestyle='--') # Stiplet linje i 0
    # plt.plot(h_steps, difference_euler, label = "Euler")
    # plt.plot(h_steps, difference_rk4, label = "RK4")

    # plt.title("Differansen mellom den analytiske og RK4 ved t = 20 s")
    # plt.xlabel('$h$')           
    # plt.ylabel(r'$\Delta\theta$ [rad]')

    # plt.legend(loc=2, shadow = True)
    # plt.show()

# plot_difference()
print("hei")

## 2b

In [ ]:
def cm_plot():
    beta = calculate_beta(np.pi*sigma/sigma_0, 5) # Guess 5 for initial beta
    
    def f(t, w):  
        '''
        Funksjon som brukes til å regne ut endringen til systemet.
        Inn:
            t: float, tid
            w: s-array, beskriver diverse av systemets fysiske egenskaper
        Ut:
            dw: endringen i w i det neste tidssteget
        
        '''
    
        delta_y_c_current = w[3] - y_C_0(beta)  

        gamma_current = 2*np.arccos(np.cos(beta/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_c_current/R)

        A = water_area(gamma_current)
       
        d_theta = w[1]
        d_omega = -((F_B(A) * h)/calculate_I_c(0, 0))*np.sin(w[0])
        d_x     = w[4]
        d_y     = w[5]
        d_v_x   = np.array([0])
        d_v_y   = (F_G(m_boat) + F_B(A))/m_boat


        dw = np.array([
            d_theta,
            d_omega,
            d_x,
            d_y,   
            d_v_x,  
            d_v_y 
        ])
        
        
        return dw

    theta_0 = np.array([np.radians(20)])
    omega_0 = np.array([np.radians(0)])
    x_C_0 = np.array([0])
    initial_y_C = np.array([y_C_0(beta)])
    v_x_0 = np.array([0])
    v_y_0 = np.array([0])



    w_0 = np.array([
        theta_0, 
        omega_0, 
        x_C_0, 
        initial_y_C, 
        v_x_0, 
        v_y_0
        ])



    t_0 = 0
    t_end = 20

    h_step = 0.01

    t_num_rk4, w_num_rk4 = solve_ODE(f, t_0, t_end, w_0, h_step, rk4_step)
    np.savez('simulation_data/2d',t_num_rk4=t_num_rk4,w_num_rk4=w_num_rk4)

    y_num_rk4 = w_num_rk4[3]
    
    plt.plot(t_num_rk4, y_num_rk4, label = "dt = 0.01")
    plt.title("y-verdi som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'Massesenterets høyde, $C_y$ [m]')   
    # plt.ylim(-0.25,0.25)        
    plt.legend(loc=3, shadow = True)
    plt.show()

# cm_plot()

## Oppg 2c



In [ ]:
def check_capsizing(w_num):
    '''
    Checks if the boat is capsizing. 
    Input:
    w_num: array with all the physical properties of the boat
    output:
    w_num: Same array, but now updated values if the boat has capsized.
    
    '''
    beta = calculate_beta(np.pi*sigma/sigma_0, 5) # Guess 5 for initial beta

    theta_arr = w_num[0]
    y_arr = w_num[3]
    for i in range(len(theta_arr)-1):
        theta_n = theta_arr[i]
        delta_y_n= y_arr[i+1]-y_arr[i]
        delta_y_c_n = y_C(beta,delta_y_n)
        if theta_n > 0.5*(np.pi-gamma(beta,theta_n,delta_y_c_n,)): #Sjekk om båten kantrer
           
            theta_arr[i+1] = np.pi/2 * np.sign(theta_arr[i])
           
            for j in range(1,(len(w_num))):
                w_num[j][i+1]=0
            
    return w_num
    

# test_arr = np.zeros((10,4))
# print(test_arr)
# print(len(test_arr[0]))
# test_arr[:,1]=1
# print(test_arr)


In [ ]:
def cm_plot_capsizing():
    beta = calculate_beta(np.pi*sigma/sigma_0, 5) # Guess 5 for initial beta
    
    def f(t, w):  
        '''
        Funksjon som brukes til å regne ut endringen til systemet.
        Inn:
            t: float, tid
            w: s-array, beskriver diverse av systemets fysiske egenskaper
        Ut:
            dw: endringen i w i det neste tidssteget
        
        '''
    
        delta_y_c_current = w[3] - y_C_0(beta)  

        gamma_current = 2*np.arccos(np.cos(beta/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_c_current/R)

        A = water_area(gamma_current)
       
        d_theta = w[1]
        d_omega = -((F_B(A) * h)/calculate_I_c(0, 0))*np.sin(w[0])
        d_x     = w[4]
        d_y     = w[5]
        d_v_x   = np.array([0])
        d_v_y   = (F_G(m_boat) + F_B(A))/m_boat


        dw = np.array([
            d_theta,
            d_omega,
            d_x,
            d_y,   
            d_v_x,  
            d_v_y 
        ])
        
        
        return dw

    theta_0 = np.array([np.radians(0)])
    omega_0 = np.array([np.radians(26)])
    x_C_0 = np.array([0])
    initial_y_C = np.array([y_C_0(beta)])
    v_x_0 = np.array([0])
    v_y_0 = np.array([0])



    w_0 = np.array([
        theta_0, 
        omega_0, 
        x_C_0, 
        initial_y_C, 
        v_x_0, 
        v_y_0
        ])



    t_0 = 0
    t_end = 20

    h_step = 0.01

    t_num_rk4, w_num_rk4 = solve_ODE(f, t_0, t_end, w_0, h_step, rk4_step)
    check_capsizing(w_num_rk4)
    np.savez('simulation_data/2c',t_num_rk4=t_num_rk4,w_num_rk4=w_num_rk4)
    theta_num_rk4 = w_num_rk4[0]
    y_num_rk4 = w_num_rk4[3]
    
    plt.plot(t_num_rk4, w_num_rk4[5], label = "dt = 0.01")
    plt.title("y-verdi som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'Massesenterets høyde, $C_y$ [m]')   
    # plt.ylim(-0.25,0.25)        
    plt.legend(loc=3, shadow = True)
    plt.show()

# cm_plot_capsizing()

In [ ]:
def load_off_boat(w_num):
    R=10
    s_L=w_num[6]
    for i in s_L:
        if abs(i)>R:
            print("Load fell off! Position number in s_L: ", i )
            m_L = 0 #denne må ha tilgang
    
    return w_num

def load_on_fence(w_num):
    R=10
    s_L=w_num[6]
    v_L = w_num[7]   
    indexes_positive = np.nonzero(s_L>R)
    indexes_negative = np.nonzero(s_L<R)
    s_L[indexes_positive] = R
    s_L[indexes_negative] = -R
    v_L[indexes] = 0 #dette blir ikke helt riktig

    w_num[6]=s_L
    w_num[7]=v_L
    return w_num



arr = np.zeros((8,13))
arr[6] = 16
arr[6,8] = -9
arr[6,9] = -16
arr[7]=3


print(arr)
print("---")
print(load_on_fence(arr))

## Oppg 2d

In [ ]:
def boat_with_load():
        
    def f(t, w):  
        '''
        Funksjon som brukes til å regne ut de nye verdiene til w-arrayet.
        Inn:
            t: float, tid
            w: s-array, beskriver diverse av systemets fysiske egenskaper
        Ut:
            w-arrayet med oppdaterte verdier
        
        '''
        m_load = 0.2*m_boat #Gjør om til array senere, evt som input til funksjonen
        m_tot = m_boat + m_load
        
        beta = calculate_beta(np.pi*sigma/sigma_0 + 2*m_load/(sigma_0 * R**2),5) #Ikke lenger samme som før
        print("beta = ", np.degrees(beta))

        A = (np.pi * R**2 *sigma + 2*m_load)/(2*sigma_0) #eller A_0?
        
        delta_y_c_current = w[3] - y_C_0(beta) 
        I_c = calculate_I_c(m_tot,)
        gamma_current = 2*np.arccos(np.cos(beta/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_c_current/R)

        # A = water_area(gamma_current) #gammel
        return np.array([w[1],    -((F_B(A) * h)/I_c)*np.sin(w[0] ), w[4], w[5], 0, (F_G(m_tot) + F_B(A))/m_tot,0,0])


    def plot_with_load():
        theta_0 = np.radians(0)
        omega_0 = np.radians(23.8) #Rundt her et sted, men sjekk med newtons metode på ligningen hvis det er mulig
        x_C_0 = 0
        v_x_0 = 0
        v_y_0 = 0

        w_0 = np.array([theta_0, omega_0, x_C_0, y_C_0, v_x_0, v_y_0,0,0])

        t_0 = 0
        t_end = 20

        h_step = 0.01

        t_num_rk4, w_num_rk4 = solve_ODE(f, t_0, t_end, w_0, h_step, rk4_step)
        w_num_rk4 = check_capsizing(w_num_rk4)
        theta_arr = w_num_rk4[:,0]
        np.savez('simulation_data/rk4_water_move',t_num_rk4=t_num_rk4,w_num_rk4=w_num_rk4)

        plt.plot(t_num_rk4, theta_arr, label = "dt = 0.01")
        plt.title("Vinkelutslag som funksjon av tid")
        plt.xlabel('$t$ [s]')           
        plt.ylabel(r'Massesenterets høyde, $C_y$ [m]')   
        # plt.ylim(-0.25,0.25)        
        plt.legend(loc=3, shadow = True)
        plt.show()
    plot_with_load()
    
# boat_with_load()

## 2f
...

In [ ]:
def wind_and_friction(F_0, k_f, omega_0_input):
    beta = calculate_beta(np.pi*sigma/sigma_0, 5) # Guess 5 for initial beta
    
    #k_f = 1000
    omega_0 = np.sqrt((m_boat*g*h)/calculate_I_c(0, 0))
    omega_w = 0.93*omega_0
    
    def f(t, w):  
        '''
        Funksjon som brukes til å regne ut endringen til systemet.
        Inn:
            t: float, tid
            w: s-array, beskriver diverse av systemets fysiske egenskaper
        Ut:
            dw: endringen i w i det neste tidssteget
        
        '''
    
        delta_y_c_current = w[3] - y_C_0(beta)  
        # print("cm_plot_c w3",w[3])
        # print("gamma innmat: ", np.cos(beta/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_c_current/R)
        gamma_current = 2*np.arccos(np.cos(beta/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_c_current/R)

        A = water_area(gamma_current)
       
        F_w = F_0*np.cos(t*omega_w)
        current_friction = friction(k_f, w[1], gamma_current)
    
        d_theta = w[1]
        d_omega = (1/calculate_I_c(0, 0))*(-F_B(A)*h*np.sin(w[0]) + F_w*w[3] + current_friction*(w[3] - (R*(np.cos(gamma_current/2) - 1))))
        d_x     = w[4]
        d_y     = w[5]
        d_v_x   = (1/m_boat)*(current_friction + F_w)
        d_v_y   = (1/m_boat)*(F_G(m_boat) + F_B(A))

        dw = np.array([
            d_theta,
            d_omega,
            d_x,
            d_y,   
            d_v_x,  
            d_v_y 
        ])
        
        
        return dw

    theta_0 = np.array([np.radians(0)])
    # omega_0 = np.array([np.radians(omega_0_imput)])
    omega_0 = np.array([omega_0_input])
    x_C_0 = np.array([0])
    initial_y_C = np.array([y_C_0(beta)])
    v_x_0 = np.array([0])
    v_y_0 = np.array([0])

    w_0 = np.array([
        theta_0, 
        omega_0, 
        x_C_0, 
        initial_y_C, 
        v_x_0, 
        v_y_0
    ])

    t_0 = 0
    # t_end = 300
    t_end = 1000

    h_step = 0.1

    t_num_rk4, w_num_rk4 = solve_ODE(f, t_0, t_end, w_0, h_step, rk4_step)
    # print(w_num_rk4[0])
    w_num_rk4 = check_capsizing(w_num_rk4)
    theta_arr = w_num_rk4[0]
    # print(theta_arr)

    np.savez('simulation_data/rk4_water_move',t_num_rk4=t_num_rk4,w_num_rk4=w_num_rk4)

    y_num_rk4 = w_num_rk4[3]
    plt.plot(t_num_rk4, theta_arr, label = "dt = 0.01")
    plt.title("y-verdi som funksjon av tid")
    plt.xlabel('$t$ [s]')           
    plt.ylabel(r'Massesenterets høyde, $C_y$ [m]')   
    # plt.ylim(-0.25,0.25)        
    plt.legend(loc=3, shadow = True)
    plt.show()

    
F_0 = 0*m_boat*g
k_f = 100
wind_and_friction(F_0, k_f, 0.4)

## 2g

In [ ]:
F_0 = 0.625*m_boat*g
k_f = 100
omega_0 = np.radians(2)

wind_and_friction(F_0, k_f, omega_0)

In [ ]:
def check_capsizing(w_num):
    '''
    Checks if the boat is capsizing. 
    Input:
    w_num: array with all the physical properties of the boat
    output:
    w_num: Same array, but now updated values if the boat has capsized.
    
    '''
    beta = calculate_beta(np.pi*sigma/sigma_0, 5) # Guess 5 for initial beta

    theta_arr = w_num[0]
    y_arr = w_num[3]
    for i in range(len(theta_arr)-1):
        theta_n = theta_arr[i]
        delta_y_n= y_arr[i+1]-y_arr[i]
        delta_y_c_n = y_C(beta,delta_y_n)
        if theta_n > 0.5*(np.pi-gamma(beta,theta_n,delta_y_c_n,)): #Sjekk om båten kantrer
           
            theta_arr[i+1] = np.pi/2 * np.sign(theta_arr[i])
           
            for j in range(1,(len(w_num))):
                w_num[j][i+1]=0
            
    return w_num

In [ ]:
def universal_func(w_start_vals, t_end_input, h_step_input, F_0, k_f, omega_w_input, m_boat_input, m_L_input):
    '''
    Universell funksjon som kan benyttes til å beregne bevegelsen til skipet for mange ulike tilfelle.
    Gjennom hva man skriver i funksjonens argument kan man bla. simulere påvikningen friksjon, bølger og last.
    Inn:
        w_start_vals: s-array, inneholder alle startverdier 
            w_start_vals = [theta_0, omega_0, x_C_0, y_C_0, v_x_0, v_y_0, s_L_0, v_L_0]
        t_end_input: float, sluttid til simulasjonen
        h_step_input: float, steglengde til simulasjonen
        F_0: float, amplitiden til den harmoniske funksjonen som beskriver vind/bølger
        k_f: float, friksjonskoeffesient
        omega_w_input: float, vinkelfrekvensen til den harmoniske funksjonen som beskriver vind/bølger
        m_boat_input: float, massen til båten
        m_L_input: float, massen til lasten
    Ut:
        t_num: array, tidsverdier
        w_num: array, w som inneholder alle beregnede verdier
    '''
    
    
    # Sector-angle in equilibrium
    beta_step = calculate_beta((np.pi*sigma*(R**2)+2*m_L)/(sigma_0*(R**2)), 5) # Guess 5 for initial beta
    m_tot = m_boat + m_L
    
    # Equating the moment of intertia
    I_C = calculate_I_c(0, 0)
    
    def f(t, w):  
        '''
        Funksjon som brukes til å regne ut endringen til systemet.
        Inn:
            t: float, tid
            w: s-array, beskriver diverse av systemets fysiske egenskaper
        Ut:
            dw: endringen i w i det neste tidssteget
        
        '''
        # Vertical displacement
        delta_y_C_step = w[3] - y_C_0(beta_step) 
        
        # Sector-angle
        gamma_step = 2*np.arccos(np.cos(beta_step/2) - (4/(3*np.pi))*(1-np.cos(w[0])) + delta_y_C_step/R)
       
        # Repressed water
        A_step = water_area(gamma_step)
        
        # Gravitation and bouancy
        F_G_step = F_G(m_boat)
        F_B_step = F_B(A_step)
        
        # Friction and wind/waves
        F_w_step = F_0*np.cos(t*omega_w)
        F_f_step = friction(k_f, w[1], gamma_step)
        
        # The components of the contact-force between the load and the ship
        F_L_x_step = m_L*g*np.cos(w[0])*np.sin(w[0])
        F_L_y_step = -m_L*g*(np.cos(w[0])**2)
 
        # Torque 
        tau_B_step = -F_B_step*h*np.sin(w[0])
        tau_f_step = F_f_step*(w[3] - (R*(np.cos(gamma_step/2)-1)))
        tau_w_step = F_w_step*w[3]
        tau_L_step = -m_L*g*np.cos(w[0])*w[6]
    
        # Equate valier for the derivative of the w-vector, and combining them in an array
        d_theta = w[1]
        d_omega = (1/I_C)*(tau_B_step + tau_f_step + tau_w_step + tau_L_step)
        d_x     = w[4]
        d_y     = w[5]
        d_v_x   = (1/m_tot)*(F_f_step + F_w_step + F_L_x_step)
        d_v_y   = (1/m_tot)*(F_G_step + F_B_step + F_L_y_step)
        d_s_L = w[7]
        d_v_L = g*np.sin(w[0])
        
        dw = np.array([
            d_theta,
            d_omega,
            d_x,
            d_y,   
            d_v_x,  
            d_v_y, 
            d_s_L,
            d_v_L
        ])
        
        return dw

    # Initsialverdier
    theta_0 = np.array([np.radians(w_start_vals[0])])
    omega_0 = np.array([w_start_vals[1]])
    x_C_0 = np.array([w_start_vals[2]])
    
    # Tenk ut en god løsning på dette. 
    initial_y_C = np.array([y_C_0(beta_step)])
    #initial_y_C = np.array([w_start_vals[3]])
    
    v_x_0 = np.array([w_start_vals[4]])
    v_y_0 = np.array([w_start_vals[5]])
    s_L_0 = np.array([w_start_vals[6]])
    v_L_0 = np.array([w_start_vals[7]])

    w_0 = np.array([
        theta_0, 
        omega_0, 
        x_C_0, 
        initial_y_C, 
        v_x_0, 
        v_y_0, 
        s_L_0, 
        v_L_0
    ])

    t_0 = 0
    t_num, w_num = solve_ODE(f, t_0, t_end_input, w_0, h_step_input, rk4_step)
    w_num = check_capsizing(w_num)
    return t_num, w_num

w_start_vals = [0, np.radians(2), 0, 0, 0, 0, 0, 0]

#F_0 = 0.625*m_boat*g
F_0 = 0
k_f = 100
t_end = 240
h_step = 0.1

omega_0 = np.sqrt((m_boat*g*h)/calculate_I_c(0, 0))
omega_w = 0.93*omega_0

m_boat = A_s*sigma
m_L = 0

t_num, w_num = universal_func(w_start_vals, t_end, h_step, F_0, k_f, omega_w, m_boat, m_L)

In [ ]:
def plot_func(t_num_arr, values_2d_arr, label_arr, title_name, x_axis_name, y_axis_name):
    for i in range(len(values_2d_arr)):
        plt.plot(t_num_arr, values_2d_arr[i], label = label_arr[i])
    plt.title(title_name)
    plt.xlabel(x_axis_name)           
    plt.ylabel(y_axis_name)   
    # plt.ylim(-0.25,0.25)        
    plt.legend(loc=3, shadow = True)
    plt.show()

#theta_arr = w_num[0]

np.savez('simulation_data/rk4_water_move',t_num_rk4=t_num,w_num_rk4=w_num)

theta_num = w_num[0]
plot_func(t_num, [theta_num], ["Test"], "Title", "x-axis", "y-axis")

## Oppgave 2h

En interresant tanke som opstår ved undersøkelse av båtens vugging er sammensettingen av bølger. ...

In [ ]:
def fourier_plot():
    #       Extract the simulation data
    sim_data = np.load('simulation_data/rk4_water_move.npz')
    t_num_rk4, w_num_rk4 = sim_data['t_num_rk4'], sim_data['w_num_rk4']

    theta_num = w_num_rk4[0]

    #       Plot theta
    plt.plot(t_num_rk4, np.rad2deg(theta_num))
    plt.ylabel("Vinkelutslag $\theta$ [$^\circ$]")
    plt.xlabel("Tid [$s$]")
    plt.show()

    ############################################################################
    #       Find the (fast) fourier transform
    N = len(theta_num)
    timestep = (t_num_rk4[-1] - t_num_rk4[0])/N

    theta_fft = fft(theta_num)
    normalized_theta_fft = 2/N * np.abs(theta_fft)

    freqs = fftfreq(N, timestep)
    

    ###############################################
    #   Plotting the fourier transform
    print(f'Numbe of freqs: {len(freqs)}')
    print(f'Ved øyemål burde frekvens_1 ligge rundt 0.38Hz, og frekvens_2 ligge rundt 0.012Hz')
    plt.bar(
        freqs,
        np.rad2deg(normalized_theta_fft),
        width = 5/N # Need a width inverse proportional to N, as this is the number of data points 
    ) 
    # plt.plot(
    #     freqs,
    #     normalized_theta_fft,
    #     'o'    
    # )    
    plt.xlim((0,  0.5))
    plt.ylabel("Gjennomsnittlig vinkelutslag $\theta$ [$^\circ$]")
    plt.xlabel("Vinkelfrekvens [Hz(?)]")
    plt.show()
    



    print(theta_num)
    # theta_fft = fft(theta_num)

fourier_plot()

In [ ]:
def fourier_test():
    ###############################################
    #   Inital system
    N_test = 128*8
    x_axis, timestep = np.linspace(0, 40, N_test, retstep=True)
    omega_1 = np.pi
    omega_2 = 1.1*np.pi

    y1 = np.sin(x_axis*omega_1)
    y2 = 0.75*np.sin(x_axis*omega_2) 
    

    period = lambda omega: (2*np.pi)/omega
    freq = lambda omega: 1/period(omega)

    # plt.plot(x_axis, y1)
    # plt.plot(x_axis, y2)
    # plt.plot(x_axis, y1 + y2)
    plt.bar(
        x_axis,
        y1+y2,
        width = 0.01    
    )
    plt.plot(x_axis, y1+y2, 'o')
    plt.show()

    print(f'Frequency 1: {freq(omega_1)}')
    print(f'Frequency 2: {freq(omega_2)}')
    print()
    print(f'N: {N_test}\n Timestep: {timestep}')



    ###############################################
    #   The fourier part


    # y_fft =  2.0/N_test * np.abs(y_fft[0:N_test//2])
    y_fft = fft(y1+y2)
    y_plot = 2/N_test * np.abs(y_fft)

    x_fft = fftfreq(N_test, d=timestep)


    ###############################################
    #   Plotting the fourier transform
    plt.bar(
        x_fft,
        y_plot,
        width = 0.01    
    )
    plt.plot(x_fft, y_plot, 'o')
    
    
    plt.xlim((0,  4))
    # print(np.sum(2.0/N_test * np.abs(y_fft[0:N_test//2])))



fourier_test()

# X.0 Conclusion

Lorem Ipsum...